### 데이터 분석 관련

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Scikit_Learn의 머신러닝 모듈들

In [2]:
# 선형회귀
from sklearn.linear_model import LogisticRegression

# 서포트 벡터 머신
from sklearn.svm import SVC, LinearSVC

# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier

# k-최근접 이웃
from sklearn.neighbors import KNeighborsClassifier

### 데이터 읽기

In [3]:
train_df = pd.read_csv("./titanic/train.csv")
test_df = pd.read_csv("./titanic/test.csv")

In [4]:
# 데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


번호는 큰 의미를 가지지않음, 이름과 티켓의 경우 불규칙성이많아 처리하기 어려움.

In [5]:
train_df.info()
print('-'*20)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    

---
test dataset = 891 // feature = 12

train dataset = 418 // feature = 11 (survived 없음)

---
### 주의

각 데이터는 빈 부분이 있는가?

    빈 부분이 있다면 drop할 것인가?
    default값으로 채워넣을 것인가?

데이터는 float64로 변환할수 있는가?

    아니라면 범주형 데이터로 만들 수 있는가?    
---

필요없는 부분이라고 생각되는 부분은 지운다.

PassengerID, 이름, 티켓을 지운다. << 이름과 티켓에서 가져올수 있는 데이터 없음

하지만 문제에서 결과물은 PassengerId, Survived 요소가 필요하므로 훈련 데이터에서만 삭제

In [6]:
train_df = train_df.drop(['PassengerId','Name','Ticket'], axis=1)
test_df = test_df.drop(['Name','Ticket'], axis=1)

### Null Data Check

In [7]:
for col in train_df.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (train_df[col].isnull().sum() / train_df[col].shape[0]))
    print(msg)

column:   Survived	 Percent of NaN value: 0.00%
column:     Pclass	 Percent of NaN value: 0.00%
column:        Sex	 Percent of NaN value: 0.00%
column:        Age	 Percent of NaN value: 19.87%
column:      SibSp	 Percent of NaN value: 0.00%
column:      Parch	 Percent of NaN value: 0.00%
column:       Fare	 Percent of NaN value: 0.00%
column:      Cabin	 Percent of NaN value: 77.10%
column:   Embarked	 Percent of NaN value: 0.22%


In [8]:
for col in test_df.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (test_df[col].isnull().sum() / test_df[col].shape[0]))
    print(msg)

column: PassengerId	 Percent of NaN value: 0.00%
column:     Pclass	 Percent of NaN value: 0.00%
column:        Sex	 Percent of NaN value: 0.00%
column:        Age	 Percent of NaN value: 20.57%
column:      SibSp	 Percent of NaN value: 0.00%
column:      Parch	 Percent of NaN value: 0.00%
column:       Fare	 Percent of NaN value: 0.24%
column:      Cabin	 Percent of NaN value: 78.23%
column:   Embarked	 Percent of NaN value: 0.00%


----

### 각 Unique한 Value에 대한 카운팅

In [9]:
train_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

#### 1. Pclass

Pclass는 실수로 바꿀수는 있는데

사실 연속적인 정보가 아님, 그래서 범주형 데이터로 인식하고 인코딩 해야한다.

그러므로 one-hot-encoding을 해야한다

In [10]:
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

pclass_train_dummies.columns = ["Pclass_1","Pclass_2","Pclass_3"]
pclass_test_dummies.columns = ["Pclass_1","Pclass_2","Pclass_3"]

train_df.drop(['Pclass'] , axis = 1 , inplace=True)
test_df.drop(['Pclass'] , axis = 1 , inplace=True)

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

Pclass의 원본을 없애고, 범주형으로 개별로 데이터가 변환됨

---

#### 2. Sex

남,여로 나뉘므로 이것도 one-hot-encoding한다.

In [11]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

---

#### 3. Age

나이는 연속적인 정보임. 큰 처리 필요없음

(카테고리화를 통해 일부 알고리즘에서 더 유용한 결과를 얻을 수 있음)

하지만 일부 데이터가 null이라서 이를 채워야함

1. 랜덤
2. 평균값
3. 중간값
4. 데이터버리기

<span style="color:gray">해당 튜토리얼에선 평균값을 채움.</span>

<span style="color:gray">데이터의 통일성을 위해 train dataset의 평균값을 train, test dataset에 채움.</span>

In [12]:
train_df["Age"].fillna(train_df["Age"].mean(), inplace=True)
test_df["Age"].fillna(train_df["Age"].mean(), inplace=True)

---

#### 4. SibSp & Panch

형제 자매와 부모를 한 가족으로 함께 처리할 수 있으나,

해당 튜토리얼에선 건드리지 않음.

---

#### 5. Fare

Fare은 탑승료임.

test 데이터셋에 1개의 데이터가 비어있음.

빈 부분을 fillna 메서드로 채움.

In [13]:
test_df["Fare"].fillna(0,inplace=True)

---

#### 6. Cabin

Cabin은 객실임.

Null이 대부분인 데이터이므로 버린다.

In [14]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

---

#### 7. Embarked

Embarked는 탑승 항구를 의미함

In [15]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [16]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

S가 대다수이고 일부 데이터가 비어있음을 알 수 있다.

빈부분은 S로 채운다.

In [17]:
train_df['Embarked'].fillna("S",inplace = True)
test_df['Embarked'].fillna("S",inplace = True)

비연결형 데이터이므로 one-hot-encoding

In [18]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S','C','Q']
embarked_test_dummies.columns = ['S','C','Q']

train_df.drop(['Embarked'], axis=1, inplace = True)
test_df.drop(['Embarked'], axis=1, inplace = True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_train_dummies)

---

### 데이터 나누기

학습용 데이터를 위해 데이터를 나눠야함.

(정보, 생존여부) 와 같은 형태로 나눈다.

In [19]:
X_train = train_df.drop("Survived",axis=1)
Y_train = train_df["Survived"]
X_test = test_df.drop("PassengerId", axis=1).copy()

---

### 머신러닝 알고리즘 적용하기

로지스틱 회귀, SVC, RandomForest, KNN 알고리즘을 각각 사용한다.

In [20]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

logreg.score(X_train, Y_train)

0.8058361391694725

In [21]:
# SVM

svc = SVC(C=15,gamma=0.01)

svc.fit(X_train,Y_train)

Y_pred = svc.predict(X_test)

svc.score(X_train, Y_train)

0.8754208754208754

In [22]:
#Random Forests

random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

0.9820426487093153

In [23]:
# KNN

knn = KNeighborsClassifier(n_neighbors = 10)

knn.fit(X_train, Y_train)

Y_pred = knn.predict(X_test)

knn.score(X_train, Y_train)

0.7665544332210998

---

## 제출용 파일 만들기

Random Forest 가 가장 좋은 결과를 냈으니,
그 결과로 Submission 파일을 만들어 제출한다.

In [24]:
# random_forest = RandomForestClassifier(n_estimators = 10000)
# random_forest.fit(X_train, Y_train)
# Y_pred = random_forest.predict(X_test)
# random_forest.score(X_train, Y_train)

# submission = pd.DataFrame({
#     "PassengerId": test_df["PassengerId"],
#     "Survived" : Y_pred
# })



# submission.to_csv('titanic.csv', index = False)